# Import Dependencies

In [1]:
import pandas as pd
import numpy as np

# Parse First Text File

In [2]:
file_path = 'llama_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100)
print(result1)
print(len(df1), total_questions)

Percentage of Questions with Executable Code: 55.425904317386234
1425 2571


In [3]:
#this needs to be rewritten
def check_topic(df):
    count = 0
    for i in range(0, len(df)):
        if df.iloc[i]['topic'] in df.iloc[i]['question']:
            count+=1
            
    return "Percentage of Executable Code that Contains Topic: " + str((count/len(df))*100)
        
    
result2 = check_topic(df1)
result2

'Percentage of Executable Code that Contains Topic: 82.52631578947368'

In [4]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 5.894736842105263


In [5]:
df1 = df1.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df1)):
    question_numbers.append(f"llama_q{i}")
df1['question_number'] = question_numbers
#df1_an = df1.sample(n=250, random_state = 42)

In [6]:
df1

,question,solution,answer,topic,question_number
0,Barbie has 500 pairs of shoes. 200 are high he...,def solution():\n #Barbie has 500 pairs of ...,150,Barbie,llama_q0
1,Black Panther can run 200 meters in 20 seconds...,def solution():\n #Black Panther can run 20...,0.2,Black Panther,llama_q1
2,A cookie baker has 2500 cookies. \nHe sells 10...,def solution():\n #The baker started with 2...,800,cookies,llama_q2
3,Iron Man has a total of 42 suits. 25 of the su...,def solution():\n #Iron Man has a total of ...,0,Iron Man,llama_q3
4,A pizza has 8 slices. 2 of the slices have mus...,def solution():\n #A pizza has 8 slices\n ...,2,pizza,llama_q4
...,...,...,...,...,...
1420,A unicorn has 100 horns. He uses 10 horns to m...,def solution():\n #A unicorn has 100 horns\...,90,unicorns,llama_q1401
1421,150 passengers are boarding a plane. 75 of the...,def solution():\n #150 passengers are board...,65,planes,llama_q1402
1422,LeBron James is a basketball player. He has 25...,def solution():\n #LeBron James has 25 poin...,18,LeBron James,llama_q1403
1423,Captain America is 2 times as strong as Thor. ...,def solution():\n #Captain America is 2 tim...,100,Captain America,llama_q1404


# Parse Second Text File

In [7]:
file_path = 'mathwell_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
print(result1)
print(len(df2), total_questions)

Percentage of Questions with Executable Code: 66.35177687809268
1475 2223


In [8]:
#Conduct significance test to see whether MATHWELL has a meaningful difference in percentage of executable code after finetuning
#compared to Llama finetuned on GSM8K with code solutions and base Llama
import numpy as np
from scipy import stats
mathwell = np.concatenate([np.ones(1475), np.zeros(2223-1475)])
fine_tuned_llama = np.concatenate([np.ones(2742), np.zeros(4347-2742)])
base_llama = np.concatenate([np.ones(1425), np.zeros(2571-1425)])

# Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(mathwell, fine_tuned_llama)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(mathwell, base_llama)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


T-statistic: 2.6196677957481587
P-value: 0.008821788654780824
T-statistic: 7.763552900356672
P-value: 1.0027870845274842e-14


In [9]:
result2 = check_topic(df2)
result2

'Percentage of Executable Code that Contains Topic: 80.8135593220339'

In [10]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 6.440677966101695


In [11]:
df2 = df2.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df2)):
    question_numbers.append(f"mathwell_q{i}")
df2['question_number'] = question_numbers
#df2_an = df2.sample(n=250, random_state = 42)

In [12]:
df2

,question,solution,answer,topic,question_number
0,The Cincinnati Bengals and the Kansas City Chi...,def solution():\n #The Bengals have 42 poin...,97,football,mathwell_q0
1,Serena Williams has 24 Grand Slam singles titl...,def solution():\n #Serena Williams has 24 G...,2,Serena Williams,mathwell_q1
2,1000 unicorns are in the forest. 200 of them a...,def solution():\n #1000 unicorns are in the...,500,unicorns,mathwell_q2
3,A T-Rex is 20 feet tall. A Brontosaurus is 10...,def solution():\n #A T-Rex is 20 feet tall\...,15.0,dinosaurs,mathwell_q3
4,The lacrosse team has 120 sticks. 20 of the st...,def solution():\n #Total sticks\n sticks...,0,lacrosse,mathwell_q4
...,...,...,...,...,...
1470,A bakery bakes 1200 cookies per day. 300 of th...,def solution():\n #The bakery bakes 1200 co...,500,cookies,mathwell_q1407
1471,Serena Williams has 23 Grand Slam titles. 12 o...,def solution():\n #Serena Williams has 23 G...,5,Serena Williams,mathwell_q1408
1472,Captain Marvel can punch 15 times a second. \n...,def solution():\n #Captain Marvel punches 1...,450,Captain Marvel,mathwell_q1409
1473,Barbie has 1200 Instagram followers. 100 new f...,def solution():\n #Barbie started with 1200...,3700,Barbie,mathwell_q1410


# Parse Third Text File

In [13]:
file_path = 'llema_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                if 'while' in solution:
                    continue
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df3 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df3)/total_questions)*100)
result1

8
Captain America was 1.0 times higher than he was wide.
60
There are at least one cookie left in the cookie jar


How long would you like to use your superpower? a. 1 day; b. 1 week a


'Percentage of Questions with Executable Code: 24.32288900690388'

In [14]:
result2 = check_topic(df3)
result2

'Percentage of Executable Code that Contains Topic: 69.97816593886463'

In [15]:
print(f"Percentage of questions that have 0 as answer: {(len(df3[df3['answer']==0])/len(df3))*100}")

Percentage of questions that have 0 as answer: 4.039301310043668


In [16]:
df3 = df3.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df3)):
    question_numbers.append(f"llema_q{i}")
df3['question_number'] = question_numbers
#df3_an = df3.sample(n=250, random_state = 42)

In [17]:
df3

,question,solution,answer,topic,question_number
0,Fred has 4 dogs. 3 of Fred's dogs have 4 legs ...,def solution():\n #Fred had 4 dogs\n dog...,15,dogs,llema_q0
1,At a high school football game there are 5000 ...,def solution():\n #Total number of fans\n ...,2500.0,cheer leading,llema_q1
2,A litter of kittens has 5 kittens. They want t...,def solution():\n #A litter of kittens has ...,10,cats,llema_q2
3,"The Buffalo Bandits have 10 attackers, 15 defe...",def solution():\n #The Buffalo Bandits have...,52,lacrosse,llema_q3
4,"Daddy wants to eat pizza, but not too much. He...",def solution():\n #A slice is 1 piece of pi...,3285,pizza,llema_q4
...,...,...,...,...,...
911,"Captain America has three bags, the red bag, t...",def solution():\n #Captain America has thre...,5,Captain America,llema_q907
912,Serena Williams won 22 Grand Slam titles. The ...,def solution():\n #Serena Williams won 22 G...,4,Serena Williams,llema_q908
913,Superman's heart beats 110 times per minute. H...,def solution():\n #Superman's heart beats 1...,1100,Superman,llema_q909
914,A bakery makes 21 cookies in 3 minutes. How ma...,def solution():\n #Number of cookies the ba...,10500.0,cookies,llema_q910


# Parse Fourth Text File

In [18]:
file_path = 'mammoth_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df4 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df4)/total_questions)*100)
result1

92
250
16
2
730
22
9
960
24
20
200
150
9
13333.333333333334
325.0
96
24
8
9
86400
16.666666666666668
3
86
10
50.0
5
1
3276
50
64
6
22.369418519393044
24
6000
20000
175
60.0
3000
300
23
300000
9
5.547001962252291
19
16.0
3
2500
5.0
18
240
1350
6.0
6
11990
1000
216.0
750
110
24
1.25
100
95
The coach can divide the team into two groups of 10 players each.
2000
5.0
6
18
37
2000.0
6.666666666666667
3.0
720
22
195
370
5
100
200
144
150
5.5
25.0
30
400
4.0
349
40
8.0
5
10.0
500
625
80
('Pikachu', 'Bulbasaur', 'Charmander')
15.0
47
600.0
2.0
70
2500.0
2
0
1600
480
13
2.0
110
1250.0
33.333333333333336
4.0
21
-1
23.323807579381203
10
40
1.15
24.0
400
21
160
0
24
5
12.5
110
53.333333333333336
4.0
40
18
20.0
10000
4
0.016666666666666666
2.6666666666666665
90
100
4000
7.5
120
240
2
30
96.0
5
36
1000
1
60
100
1.8181818181818181
1000
10.0
37
10
3.0
12
10.0
40.0
8
16777216
1.4
3
5
20
800
20
120
50
51
48.0
5.596
25.0
400
6
20
24
150.0
2.3333333333333335
2
40
37
24
32
10.5
7
22
150
600000
300
1600
120
6

'Percentage of Questions with Executable Code: 6.885572139303482'

In [19]:
result2 = check_topic(df4)
result2

'Percentage of Executable Code that Contains Topic: 86.9942196531792'

In [20]:
print(f"Percentage of questions that have 0 as answer: {(len(df4[df4['answer']==0])/len(df4))*100}")

Percentage of questions that have 0 as answer: 2.8901734104046244


In [21]:
df4 = df4.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df4)):
    question_numbers.append(f"mammoth_q{i}")
df4['question_number'] = question_numbers
#df4_an = df4.sample(n=250, random_state = 42)

In [22]:
df4

,question,solution,answer,topic,question_number
0,Iron Man has 23 suits of armor. He builds 12 m...,def solution():\n #Iron Man has 23 suits of...,31,Iron Man,mammoth_q0
1,Naruto has 100 ninja tools. 20 of them are shu...,def solution():\n #Naruto has 100 ninja too...,50,Naruto,mammoth_q1
2,"Thor, the God of Thunder, can throw his hammer...",def solution():\n #Thor can throw his hamme...,166.666667,"Thor, the God of Thunder",mammoth_q2
3,Iron Man can fly 100 miles per minute. \nHow m...,def solution():\n #Iron Man can fly 100 mil...,500,Iron Man,mammoth_q3
4,Patrick Mahomes is the starting quarterback fo...,def solution():\n #Number of touchdowns thr...,92,Patrick Mahomes,mammoth_q4
...,...,...,...,...,...
341,Naruto has 12 ramen. He eats 3 of them on each...,def solution():\n #Naruto started with 12 r...,6,Naruto,mammoth_q341
342,"If Barbie has 245 1s, 225 5s, 123 10s, and 124...",def solution():\n #Barbie has 245 1s\n o...,5080,candy,mammoth_q342
343,The soccer team has 32 players. Each player ha...,def solution():\n #Number of players\n p...,64,soccer,mammoth_q343
344,Sasuke ate 18 cookies on Monday. He ate 20 coo...,def solution():\n #Sasuke ate 18 cookies on...,60,cookies,mammoth_q344


# Concat DFs

df = pd.concat([df1, df2, df3, df4])
df = df.sample(frac = 1, random_state = 42)
df_an = pd.concat([df1_an, df2_an, df3_an, df4_an])
df_an = df_an.sample(frac=1, random_state = 42)
#df.to_csv("data\model_questions.csv")
#df_an.to_csv("data\zooniverse_questions.csv")

df

df_an

# Send Each Question to .txt file for Zooniverse Upload

for i in range(0, len(df_an)):
    question_number = df_an.iloc[i]['question_number']
    question = df_an.iloc[i]['question']
    solution = df_an.iloc[i]['solution']
    answer = str(df_an.iloc[i]['answer'])
    topic = df_an.iloc[i]['topic']
    file_name = f"{question_number}.txt"
    file_path = "Zooniverse/final_data/"+file_name
    text = "Topic: "+ topic + "\n\nQuestion:\n" + question + "\n\nSolution:\n" + solution + "\n\nAnswer: " + answer
    with open(file_path, "w") as f:  # Open the file in append mode ("a")
        f.write(text)  # Append the newly generated text to the file